In [ ]:
import sys
sys.path.append("../code")

In [ ]:
text = """Sub-module available for the above is sent_tokenize.
            An obvious question in your mind would be why sentence tokenization is needed when we have the option of word tokenization. 
            Imagine you need to count average words per sentence, how you will calculate? 
            For accomplishing such a task, you need both sentence tokenization as well as words to calculate the ratio. 
            Such output serves as an important feature for machine training as the answer would be numeric. 
            Check the below example to learn how sentence tokenization is different from words tokenization.
            The taxes the President announced will not lower work incentives. Evrika!
            John can’t keep up with Mary’s rapid mood swings.
            """

In [ ]:
from utils import tokenize_into_sentences, filter_sentences, preprocess, UsedRoles
from word_embedding import run_word2vec, compute_embedding, USE, SIF_Word2Vec
from semantic_role_labeling import SRL, extract_roles, postprocess_roles
from clustering import Clustering
from sklearn.cluster import KMeans
from cooccurrence import build_df, CoOccurrence

In [ ]:
used_roles=UsedRoles()
used_roles['ARG2']=True
print(f"{used_roles.used}\n{used_roles.embeddable}\n{used_roles.not_embeddable}\n")

In [ ]:
srl = SRL("./srl-model-2018.05.25.tar.gz")
srl([" ".join(["What","are","you","doing?"])])

In [ ]:
srl = SRL("./bert-base-srl-2020.03.24.tar.gz")
srl([" ".join(["What","are","you","doing"])])


In [ ]:
srl = SRL("./srl-model-2018.05.25.tar.gz")
srl(["the government increases spending without raising taxes."])


In [ ]:
srl = SRL("./bert-base-srl-2020.03.24.tar.gz")
srl(["the government increases spending without raising taxes."])

In [ ]:
srl = SRL("./srl-model-2018.05.25.tar.gz")


In [ ]:
use = USE('./USE-4')
use(["What","are","you","doing"]).shape

In [ ]:
sif_w2v = SIF_Word2Vec("./nytimes_word2vec.model")
sif_w2v(["what","are","you","doing"]).shape

In [ ]:
kmeans=KMeans(random_state=0)

In [ ]:
sentences = tokenize_into_sentences(text)
sentences

In [ ]:
sentences = filter_sentences(sentences, max_sentence_length=350)
sentences

In [ ]:
srl_res = srl(sentences=sentences)
srl_res

In [ ]:
roles,sentence_index = extract_roles(srl_res)
sentence_index

In [ ]:
roles

In [ ]:
postproc_roles = postprocess_roles(roles)
postproc_roles

In [ ]:
sif_vectors, sif_statements_index, sif_funny_index =compute_embedding(sif_w2v,statements=postproc_roles,
                                                                      used_roles=used_roles)

In [ ]:
sif_statements_index

In [ ]:
{el:sif_vectors[el].shape for el in sif_vectors.keys()}

In [ ]:
sif_funny_index

In [ ]:
postproc_roles[0]["ARG2"]

In [ ]:
USE_vectors, USE_statements_index, USE_funny_index = compute_embedding(use,roles,used_roles)


In [ ]:
USE_statements_index

In [ ]:
{el:USE_vectors[el].shape for el in USE_vectors.keys()}

In [ ]:
USE_funny_index

In [ ]:
clustering = Clustering(cluster=kmeans,n_clusters={'ARGO':2, 'ARG1': 1, 'ARG2':1, 'B-V':2},
                         used_roles=used_roles)

In [ ]:
clustering.fit(vectors=sif_vectors)

In [ ]:
{el:clustering._cluster[el].labels_ for el in clustering._cluster.keys()}

In [ ]:
clustering_res = clustering.predict(vectors=sif_vectors)
clustering_res

In [ ]:
labels = clustering.label_most_similar_in_w2v(sif_w2v)
labels

In [ ]:
df = build_df(
    clustering_res=clustering_res,
    postproc_roles=postproc_roles,
    statement_index=sif_statements_index,
    used_roles=used_roles,
)

In [ ]:
df

In [ ]:
labels

In [ ]:
cooc = CoOccurrence(df, labels, used_roles)

In [ ]:
cooc.subset=None ## by convention None means take all roles
cooc.subset

In [ ]:
cooc.narratives_counts

In [ ]:
cooc.narratives_pmi

In [ ]:
cooc.subset={"ARGO","ARG1","B-V","B-ARGM-MOD","B-ARGM-NEG"}
print(cooc.normal_order)
cooc.display_order

In [ ]:
cooc.narratives_counts

In [ ]:
cooc.narratives_pmi

In [ ]:
cooc.subset={"ARGO","ARG1","B-V"}
cooc.narratives_counts

In [ ]:
cooc.narratives_pmi